In [1]:
#imports
import os
from glob import glob

In [4]:
#helper function
def load_file(_output_format_file):
    _output_dict = {}
    _fid = open(_output_format_file, 'r')
    # next(_fid)
    for _line in _fid:
        _words = _line.strip().split(',')
        _frame_ind = int(_words[0])
        if _frame_ind not in _output_dict:
            _output_dict[_frame_ind] = []
        if len(_words) == 5: #polar coordinates 
            _output_dict[_frame_ind].append([int(_words[1]), int(_words[2]), float(_words[3]), float(_words[4])])
        elif len(_words) == 6: # cartesian coordinates
            _output_dict[_frame_ind].append([int(_words[1]), int(_words[2]), float(_words[3]), float(_words[4]), float(_words[5])])
    _fid.close()
    return _output_dict


def get_zero_and_one_event(events):
    miss_frame = 0
    one_frame = 0
    for frame in range(max(events.keys())+1):
        if frame not in events:
            miss_frame+=1
        elif len(events[frame])==1:
            one_frame+=1
    return miss_frame, one_frame, max(events.keys())

def get_cons_zero(events):
    max_cons = 0
    cons_0 = 0
    for frame in range(max(events.keys())+1):
        if frame not in events:
            cons_0+=1
        else:
            max_cons = max(max_cons, cons_0)
            cons_0 = 0
    return max_cons

def get_cons_ov(events):
    max_cons = 0
    cons_ov = 0
    for frame in range(max(events.keys())+1):
        if (frame in events) and (len(events[frame])>1):
            cons_ov+=1
        else:
            max_cons = max(max_cons, cons_ov)
            cons_ov = 0
    return max_cons

In [3]:
#variables/constants
base_dir = '/vast/sk8974/experiments/dsynth/data/util_data/stars22-23/metadata_dev'
train_files = glob(f'{base_dir}/*train*/*.csv')
test_files = glob(f'{base_dir}/*test*/*.csv')
names = ['train', 'test']
folds = [train_files, test_files]

### Calculate silences and nonOv

In [12]:
for fold, name in zip(folds, names):
    total_zero, total_one, total_frame = 0,0,0
    for file in fold:
        data = load_file(file)
        zero, one, frame = get_zero_and_one_event(data)
#         print(zero, one, frame)

        total_zero+=zero
        total_one+=one
        total_frame+=frame
        
    print(f'{name} 0s:{round(total_zero*100/total_frame,2)}%, 1s:{round(total_one*100/total_frame,2)}%')

train 0s:16.24%, 1s:52.7%
test 0s:10.39%, 1s:55.2%


### longest 0s and overlaps

In [7]:
for fold, name in zip(folds, names):
    max_0, max_ov = 0, 0
    for file in fold:
        data = load_file(file)
        this_max_0 = get_cons_zero(data)
        this_max_ov = get_cons_ov(data)
        max_0 = max(max_0, this_max_0)
        max_ov = max(max_ov, this_max_ov)        
        
    print(f'{name} 0s:{max_0}, oVs:{max_ov}')

train 0s:495, oVs:3838
test 0s:163, oVs:1624


### Plot audios